In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/Kacper-M-Michalik/splatter-image

Cloning into 'splatter-image'...
remote: Enumerating objects: 391, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 391 (delta 149), reused 124 (delta 106), pack-reused 185 (from 2)
Receiving objects: 100% (391/391), 3.02 MiB | 56.31 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [ ]:
!pip install -r splatter-image/requirements.txt

In [ ]:
!pip install datasets

In [ ]:
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 1053, done.
remote: Total 1053 (delta 0), reused 0 (delta 0), pack-reused 1053 (from 1)
Receiving objects: 100% (1053/1053), 78.70 MiB | 11.56 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects: 100% (17

In [ ]:
%cd /content/gaussian-splatting
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.0;7.5;8.0+PTX"
!pip install -e submodules/diff-gaussian-rasterization
%cd ..

/content/gaussian-splatting
Obtaining file:///content/gaussian-splatting/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for diff_gaussian_rasterization
/content


Running LoRA adapters training

In [ ]:
!python /content/splatter-image/train_network_lora.py \
    +dataset=cars_priors \
    opt.pretrained_hf=True \
    data.subset=-1 \
    logging.ckpt_iterations=1000 \
    logging.val_log=1000 \
    opt.iterations=10000 \
    opt.lora_finetune=True \
    +opt.lora_rank=32 \
    +opt.lora_alpha=32 \
    +opt.lora_dropout=0.05 \
    hydra.run.dir=/content/drive/MyDrive/Splatter_Experiments/lora_Per5

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

wandb: (

*Running LoRA Evaluation*

In [ ]:
!python splatter-image/eval_lora.py \
    /content/drive/MyDrive/Splatter_Experiments/lora_r8a8d1_full/model_best.pth \
    /content/drive/MyDrive/EVAL_r8a8d01BestTests \
    --dataset_override cars_priors \
    --save_vis 1


[LoRA Evaluation] Config: default_config | Dataset: cars_priors
>> Model Architecture Mode: Priors Enabled
>> Building LoRA-compatible architecture...
/content/splatter-image/scene/gaussian_predictor_lora.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.weight = nn.Parameter(torch.tensor(weight_init([out_channels, in_channels, kernel, kernel], **init_kwargs) * init_weight, dtype=torch.float32)) if kernel else None
/content/splatter-image/scene/gaussian_predictor_lora.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.bias = nn.Parameter(torch.tensor(weight_init([out_channels], **init_kwargs) * init_bias, dtype=torch.float32)) if kernel and bias else None
>> Loading wei